In [ ]:
#| default_exp structs

In [ ]:
#|export

from flax.struct import dataclass
import jax
import jax.numpy as jnp
from typing import Optional, Tuple, List

@dataclass(frozen=True)
class GeneticMap:
    """
    Defines the genetic map of the organism.

    Attributes:
        chromosome_lengths (jnp.ndarray): The genetic length of each chromosome in Morgans.
            Shape: (n_chr,)
        locus_positions (List[jnp.ndarray]): A list where each element is a JAX array
            containing the Morgan positions of loci for a single chromosome.
    """
    chromosome_lengths: jnp.ndarray
    locus_positions: List[jnp.ndarray]


@dataclass(frozen=True)
class Population:
    """
    Represents a collection of individuals within a breeding program.

    Attributes:
        geno (jnp.ndarray): Genotypes.
            Shape: (n_individuals, n_chr, 2, n_loci)
        ibd (jnp.ndarray): Identity-by-descent tracking for founder alleles.
            Shape: (n_individuals, n_chr, 2, n_loci)
        meta (jnp.ndarray): Metadata [id, mother_id, father_id, birth_gen].
            Shape: (n_individuals, 4)
    """
    geno: jnp.ndarray
    ibd: jnp.ndarray
    meta: jnp.ndarray

@dataclass(frozen=True)
class Trait:
    """
    Defines the genetic architecture of one or more traits.

    This structure holds the information linking genotypes to phenotypes,
    based on an additive and dominance QTL model.

    Attributes:
        qtl_chromosome (jnp.ndarray): Chromosome index for each QTL.
            Shape: (n_qtl,)
        qtl_position (jnp.ndarray): Locus index for each QTL within its chromosome.
            Shape: (n_qtl,)
        qtl_effects (jnp.ndarray): The additive effect of each QTL on each trait.
            Shape: (n_qtl, n_traits)
        qtl_dominance_effects (jnp.ndarray): The dominance effect of each QTL on each trait.
            Shape: (n_qtl, n_traits)
        intercept (jnp.ndarray): The base value for each trait.
            Shape: (n_traits,)
    """
    qtl_chromosome: jnp.ndarray
    qtl_position: jnp.ndarray
    qtl_effects: jnp.ndarray
    qtl_dominance_effects: jnp.ndarray
    intercept: jnp.ndarray


@dataclass(frozen=True)
class BreedingState:
    """
    The complete, self-contained state of a single simulation replicate at a point in time.

    This object is the primary carrier passed between steps in the simulation.

    Attributes:
        population (Population): The current population of individuals.
        key (jax.Array): The JAX pseudo-random number generator key.
        generation (int): The current generation number of the simulation.
    """
    population: Population
    key: jax.Array
    generation: int
    next_id: int # To ensure unique IDs for new individuals


In [ ]:
#|export

import jax
import jax.numpy as jnp
from typing import Tuple



def quick_haplo(
    key: jax.Array,
    n_ind: int,
    n_chr: int,
    seg_sites: int,
    inbred: bool = False,
    chr_length: float = 1.0,
) -> Tuple[Population, GeneticMap]:
    """Create a simple founder population and a uniform genetic map."""
    if inbred:
        haplotypes = jax.random.randint(
            key, (n_ind, n_chr, 1, seg_sites), 0, 2, dtype=jnp.int8
        )
        geno = jnp.broadcast_to(haplotypes, (n_ind, n_chr, 2, seg_sites))
    else:
        geno = jax.random.randint(
            key, (n_ind, n_chr, 2, seg_sites), 0, 2, dtype=jnp.int8
        )

    # IBD labels: unique per haplotype across individuals
    ibd = jnp.broadcast_to(
        jnp.arange(n_ind * 2, dtype=jnp.int32).reshape(n_ind, 1, 2, 1),
        (n_ind, n_chr, 2, seg_sites),
    )

    # Meta (int32 all the way, stack doesn't accept dtype= kwarg)
    meta = jnp.stack(
        [
            jnp.arange(n_ind, dtype=jnp.int32),               # id
            jnp.full((n_ind,), -1, dtype=jnp.int32),          # mother_id
            jnp.full((n_ind,), -1, dtype=jnp.int32),          # father_id
            jnp.zeros((n_ind,), dtype=jnp.int32),             # birth_gen
        ],
        axis=-1,
    )

    population = Population(geno=geno, ibd=ibd, meta=meta)

    # Genetic map: same number of loci per chromosome, uniform spacing
    chromosome_lengths = jnp.full((n_chr,), chr_length, dtype=jnp.float32)
    locus_positions = jnp.stack(
        [jnp.linspace(0.0, chr_length, seg_sites, dtype=jnp.float32) for _ in range(n_chr)],
        axis=0,
    )
    genetic_map = GeneticMap(
        chromosome_lengths=chromosome_lengths, locus_positions=locus_positions
    )
    return population, genetic_map

#| export

def msprime_pop(
    key: jax.random.PRNGKey,
    n_ind: int,
    n_loci_per_chr: int,
    n_chr: int,
    max_pop_size: int,
    ploidy: int = 2,
    effective_population_size: int = 10_000,
    mutation_rate: float = 2e-8,
    recombination_rate_per_chr: float = 2e-8,
    maf_threshold: float = 0.1,
    num_simulated_individuals: int = None,
    base_chr_length: int = 1_000_000,
    enforce_founder_maf: bool = True
) -> Tuple[Population, jnp.ndarray]:
    """
    Creates a new, padded founder population using msprime coalescent simulation.

    Args:
        key: JAX random key.
        n_ind: Number of founder individuals to generate.
        n_loci_per_chr: Number of SNPs (loci) to select per chromosome.
        n_chr: Number of chromosomes.
        max_pop_size: The total size of the arrays to pre-allocate for JAX compatibility.
        ... (other args remain the same) ...

    Returns:
        A tuple containing:
        - A new Population object with `n_ind` active founders in padded arrays of size `max_pop_size`.
        - A JAX array representing the genetic map.
    """
    # --- Parameter Validation ---
    if n_ind > max_pop_size:
        raise ValueError(f"Number of founders ({n_ind}) cannot exceed max_pop_size ({max_pop_size}).")
        
    if effective_population_size > 100_000:
        raise ValueError(f"Effective population size {effective_population_size} is very large and may cause memory issues.")
    
    if num_simulated_individuals is None:
        multiplier = 5 if enforce_founder_maf else 2
        num_simulated_individuals = min(max(n_ind * multiplier, 1000), 10_000)
        
    if n_ind > num_simulated_individuals:
        raise ValueError(f"Number of founders requested ({n_ind}) cannot exceed the base simulated population size ({num_simulated_individuals}).")

    # --- Simulation Setup (Unchanged) ---
    key, seed_key, sex_key, numpy_seed_key = jax.random.split(key, 4)
    random_seed = int(jnp.sum(seed_key))
    numpy_seed = int(jnp.sum(numpy_seed_key))
    rng = default_rng(numpy_seed)

    chromosome_lengths = [base_chr_length] * n_chr
    num_haplotypes = num_simulated_individuals * ploidy
    rate_map_positions = [0] + list(np.cumsum(chromosome_lengths))
    rate_map_rates = [recombination_rate_per_chr] * len(chromosome_lengths)
    rate_map = msprime.RateMap(position=rate_map_positions, rate=rate_map_rates)

    ts = msprime.sim_ancestry(
        samples=num_haplotypes,
        population_size=effective_population_size,
        recombination_rate=rate_map,
        random_seed=random_seed
    )
    mts = msprime.sim_mutations(ts, rate=mutation_rate, random_seed=random_seed)

    # --- Data Extraction (Unchanged) ---
    true_num_individuals = mts.num_samples // ploidy
    founder_indices = np.sort(rng.choice(true_num_individuals, n_ind, replace=False))
    
    all_variants = list(mts.variants())
    genetic_map = np.full((n_chr, n_loci_per_chr), np.nan)
    founder_haplotype_matrix = np.full((n_ind, n_chr, ploidy, n_loci_per_chr), np.nan)

    for i in range(n_chr):
        chr_start, chr_end = rate_map.left[i], rate_map.right[i]
        chromosome_snps = [v for v in all_variants if chr_start <= v.site.position < chr_end and len(v.alleles) == 2]

        if enforce_founder_maf:
            eligible_snps = [
                var for var in chromosome_snps 
                if min(np.mean(var.genotypes.reshape(true_num_individuals, ploidy)[founder_indices]), 
                       1 - np.mean(var.genotypes.reshape(true_num_individuals, ploidy)[founder_indices])) > maf_threshold
            ]
        else:
            eligible_snps = [var for var in chromosome_snps if min(np.mean(var.genotypes), 1 - np.mean(var.genotypes)) > maf_threshold]

        num_found = len(eligible_snps)
        num_to_select = min(num_found, n_loci_per_chr)

        if num_to_select > 0:
            selected_indices = rng.choice(len(eligible_snps), num_to_select, replace=False)
            selected_snps = [eligible_snps[i] for i in selected_indices]
            selected_snps.sort(key=lambda v: v.site.position)

            for snp_idx, snp in enumerate(selected_snps):
                all_genotypes = snp.genotypes.reshape(true_num_individuals, ploidy)
                founder_genotypes = all_genotypes[founder_indices]
                founder_haplotype_matrix[:, i, :, snp_idx] = founder_genotypes
            
            positions_cm = [(v.site.position - chr_start) * recombination_rate_per_chr * 100 for v in selected_snps]
            genetic_map[i, :num_to_select] = positions_cm

    # --- Padding and Final Population Assembly ---
    n_pad = max_pop_size - n_ind

    # Convert to JAX arrays before padding
    founder_geno = jnp.array(founder_haplotype_matrix, dtype=jnp.uint8)
    gen_map_jax = jnp.array(genetic_map)

    # Pad core genotype/IBD arrays
    padded_geno = jnp.pad(founder_geno, ((0, n_pad), (0, 0), (0, 0), (0, 0)), constant_values=0)
    
    # Create and pad IBD data
    n_founder_alleles = n_ind * n_chr * ploidy * n_loci_per_chr
    founder_ids_ibd = jnp.arange(n_founder_alleles, dtype=jnp.uint32)
    founder_ibd = founder_ids_ibd.reshape(n_ind, n_chr, ploidy, n_loci_per_chr)
    padded_ibd = jnp.pad(founder_ibd, ((0, n_pad), (0, 0), (0, 0), (0, 0)), constant_values=-1)

    # Pad pedigree and identifier arrays
    padded_id = jnp.pad(jnp.arange(n_ind), (0, n_pad), constant_values=-1)
    padded_sex = jnp.pad(jax.random.choice(sex_key, jnp.array([0, 1], dtype=jnp.int8), (n_ind,)), (0, n_pad), constant_values=-1)
    
    # Create the is_active mask
    is_active = jnp.arange(max_pop_size) < n_ind

    pop = Population(
        geno=padded_geno,
        ibd=padded_ibd,
        id=padded_id,
        iid=jnp.arange(max_pop_size, dtype=jnp.int32),
        mother=jnp.full(max_pop_size, -1, dtype=jnp.int32),
        father=jnp.full(max_pop_size, -1, dtype=jnp.int32),
        sex=padded_sex,
        gen=jnp.zeros(max_pop_size, dtype=jnp.int32),
        is_active=is_active,
        pheno=jnp.full((max_pop_size, 0), jnp.nan),
        fixEff=jnp.zeros(max_pop_size, dtype=jnp.float32),
        bv=jnp.full((max_pop_size, 0), jnp.nan),
    )

    return pop, gen_map_jax


In [ ]:
#|export

import jax.numpy as jnp
import jax

@jax.jit
def compute_dosage(population: Population) -> jnp.ndarray:
    """
    Computes the dosage of the '1' allele for each individual at each locus.

    Dosage is calculated as the sum of alleles across the two homologous
    chromosomes, resulting in values of 0, 1, or 2.

    Args:
        population: A `Population` object.

    Returns:
        A 3D array of dosages with shape (n_individuals, n_chr, n_loci).
    """
    # geno shape: (n_individuals, n_chr, 2, n_loci)
    # Sum across the chromosome pair axis (axis=2)
    return jnp.sum(population.geno, axis=2)


def _flatten_gather_chr_locus(dosage_chr_locus: jnp.ndarray,
                              chr_idx: jnp.ndarray,
                              locus_idx: jnp.ndarray) -> jnp.ndarray:
    """
    dosage_chr_locus: (n_ind, n_chr, n_loci)
    chr_idx, locus_idx: (n_qtl,)
    returns: (n_ind, n_qtl) gathered pairwise along (chr,locus).
    """
    n_chr, n_loci = dosage_chr_locus.shape[1], dosage_chr_locus.shape[2]
    flat = dosage_chr_locus.reshape(dosage_chr_locus.shape[0], n_chr * n_loci)
    flat_ids = chr_idx * n_loci + locus_idx
    return jnp.take(flat, flat_ids, axis=1)





def add_trait(
    key: jax.Array,
    founder_pop: Population,
    n_qtl_per_chr: int,
    mean: jnp.ndarray,      # (n_traits,)
    var_a: jnp.ndarray,     # Additive variance (n_traits,)
    var_d: jnp.ndarray,     # Dominance variance (n_traits,)
    sigma: jnp.ndarray,     # (n_traits, n_traits) PSD
) -> Trait:
    """Sample QTLs and multi-trait effects for both additive and dominance components."""
    key, qtl_key, effect_key, dom_effect_key = jax.random.split(key, 4)

    n_chr = founder_pop.geno.shape[1]
    n_loci_per_chr = founder_pop.geno.shape[3]
    n_traits = int(mean.shape[0])
    n_total_qtl = n_qtl_per_chr * n_chr

    # --- 1. Sample QTL locations (same for additive and dominance) ---
    all_loci_indices = jnp.arange(n_chr * n_loci_per_chr, dtype=jnp.int32)
    qtl_loc_flat = jax.random.choice(qtl_key, all_loci_indices, (n_total_qtl,), replace=False)
    qtl_chromosome, qtl_position = jnp.divmod(jnp.sort(qtl_loc_flat), n_loci_per_chr)

    # --- 2. Sample and correlate raw effects ---
    cholesky_factor = jnp.linalg.cholesky(sigma.astype(jnp.float32))
    
    # Additive effects
    raw_add_effects = jax.random.normal(effect_key, (n_total_qtl, n_traits), dtype=jnp.float32)
    add_effects = raw_add_effects @ cholesky_factor.T
    
    # Dominance effects
    raw_dom_effects = jax.random.normal(dom_effect_key, (n_total_qtl, n_traits), dtype=jnp.float32)
    dom_effects = raw_dom_effects @ cholesky_factor.T

    # --- 3. Compute founder dosages and scale effects ---
    founder_dosage_full = jnp.sum(founder_pop.geno, axis=2, dtype=jnp.int32)
    qtl_dosage = _flatten_gather_chr_locus(founder_dosage_full, qtl_chromosome, qtl_position).astype(jnp.float32)
    
    # Scale additive effects
    add_gvs = qtl_dosage @ add_effects
    scale_a = jnp.sqrt(var_a.astype(jnp.float32) / (jnp.var(add_gvs, axis=0) + 1e-8))
    final_add_effects = add_effects * scale_a

    # Scale dominance effects
    # Dominance dosage is 1 for heterozygotes (additive dosage is 1), and 0 otherwise.
    # This is a clean, vectorized way to compute it.
    dominance_qtl_dosage = (qtl_dosage == 1).astype(jnp.float32)
    dom_gvs = dominance_qtl_dosage @ dom_effects
    scale_d = jnp.sqrt(var_d.astype(jnp.float32) / (jnp.var(dom_gvs, axis=0) + 1e-8))
    final_dom_effects = dom_effects * scale_d
    
    # --- 4. Calculate intercept based on total genetic value ---
    # The intercept should center the total genetic value (A + D) in the founder population.
    total_gvs = (qtl_dosage @ final_add_effects) + (dominance_qtl_dosage @ final_dom_effects)
    intercept = mean.astype(jnp.float32) - jnp.mean(total_gvs, axis=0)

    return Trait(
        qtl_chromosome=qtl_chromosome.astype(jnp.int32),
        qtl_position=qtl_position.astype(jnp.int32),
        qtl_effects=final_add_effects,
        qtl_dominance_effects=final_dom_effects,
        intercept=intercept,
    )


In [ ]:
key = jax.random.PRNGKey(42)
founder_pop = quick_haplo(
key=key,
n_ind=10,
n_chr=3,
seg_sites=100,
inbred=False
 )

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()